In [ ]:
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.preprocessing import StandardScaler

In [12]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train = train.loc[train['GrLivArea'] < 4500] # outlier removal

for df in [train, test]:

    df.set_index("Id", inplace=True)
    df['KitchenQual'] = df['KitchenQual'].fillna('TA')
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    df['BsmtFullBath'] = df['BsmtFullBath'].fillna(0)
    df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(0)
    df['GarageArea'] = df['GarageArea'].fillna(0)
    df['TotalBsmtSF'] = df['TotalBsmtSF'].fillna(0)
    df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] +df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])
    df['FullBath']  = df['FullBath'] + df['BsmtFullBath']
    df['HalfBath'] = df['HalfBath'] + df['BsmtHalfBath']
    
    df['TotSF'] = df['GrLivArea']-df['LowQualFinSF'] + df['TotalBsmtSF']

    df.drop(['FireplaceQu', 'Street', 'Utilities', 'LandContour', 
        'MasVnrType', 'Condition2', 'PoolArea', 'LotFrontage', 'CentralAir',
        'Functional', 'LandSlope', 'LotConfig', 'Fence', 'BldgType', 'Street',
        'Electrical', 'Alley', 'RoofStyle', 'KitchenAbvGr', 'BsmtFinType2',
        'Heating', 'PavedDrive', 'LandContour', 'Condition1', 'GarageCond', 
        'ExterCond', 'MSZoning', 'MiscFeature', 'SaleCondition', 'BsmtFinSF2',
        'SaleType', 'BsmtCond', 'MiscVal', 'GarageQual','EnclosedPorch',
        '3SsnPorch', 'RoofMatl', 'ScreenPorch', 'LowQualFinSF', 'Condition2',
        'PoolQC', 'PoolArea', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtQual',
        'BsmtExposure','BsmtFinType1', 'BsmtFinSF1', 'BsmtUnfSF', '1stFlrSF', 
        '2ndFlrSF', 'MSSubClass', 'LotShape', 'Neighborhood', 'YearRemodAdd',
        'Exterior1st', 'Exterior2nd', 'GarageType', 'GarageYrBlt', 'Foundation',
        'GarageFinish', 'GarageCars','GrLivArea','TotalBsmtSF','OpenPorchSF',\
             'EnclosedPorch','3SsnPorch','WoodDeckSF','ScreenPorch',\
            'TotalBsmtSF'], axis=1, inplace=True) 

    


train.to_csv('train_clean.csv', index=False)
test.to_csv('test_clean.csv', index=False)





In [13]:
test_df = pd.read_csv('test_clean.csv')

# Dummify
HS_dummy = pd.get_dummies(test_df['HouseStyle'], prefix='HS', prefix_sep='__', drop_first = True)
test_df = pd.concat([test_df.drop('HouseStyle', axis=1), HS_dummy], axis=1)
EQ_dummy = pd.get_dummies(test_df['ExterQual'], prefix='EQ', prefix_sep='__', drop_first = True)
test_df = pd.concat([test_df.drop('ExterQual', axis=1), EQ_dummy], axis=1)
HQC_dummy = pd.get_dummies(test_df['HeatingQC'], prefix='HQC', prefix_sep='__',drop_first = True)
test_df = pd.concat([test_df.drop('HeatingQC', axis=1), HQC_dummy], axis=1)
KQ_dummy = pd.get_dummies(test_df['KitchenQual'], prefix='KQ', prefix_sep='__',drop_first = True)
test_df = pd.concat([test_df.drop('KitchenQual', axis=1), KQ_dummy], axis=1)

# Extra dropped cols after Lasso regression in train data
test_df.drop(['HS__1.5Unf','HQC__Po','EQ__Gd','HS__1Story'],axis=1,inplace=True)

# Scale testing dataframe
ss = StandardScaler()
test_df = pd.DataFrame( ss.fit_transform( test_df ), columns = test_df.columns )



In [28]:
# Load Trained Model
estimator = load('elastic_net.joblib')

# Predict and reverse Log transform
predictions = np.expm1(estimator.predict(test_df))

# Save to DataFrame
index = range(1461,(1461+len(predictions)))
predictions = pd.DataFrame(data=predictions, index=index, columns=['SalePrice'])

In [31]:
# Save Predictions to csv
predictions.to_csv("kaggle_submission1.csv")

In [32]:
predictions

,SalePrice
1461,127791.175808
1462,152241.519438
1463,167502.589304
1464,198051.841272
1465,184405.720767
1466,168624.285396
1467,185455.721803
1468,159200.756471
1469,186571.044723
1470,110816.554260


NameError: name 'library' is not defined